In [1]:
import pandas as pd
import numpy as np
import os

from config import BASE_DATOS_DIR, NAME_DATA_DIR, MATRICES_DIR

In [2]:
# cargar datos
df=pd.read_csv(BASE_DATOS_DIR)
df_nombres=pd.read_csv(NAME_DATA_DIR)

df.head()

,ENTIDAD,MUNICIPIO,CODIGO,ID_ESTRATO,H001A
0,14,0,NaN,NaN,2004175.0
1,14,0,NaN,1.0,768539.0
2,14,0,NaN,2.0,346697.0
3,14,0,NaN,3.0,337722.0
4,14,0,NaN,4.0,551217.0


In [3]:
df_nombres.head()

,CVE_MUN,NOM_MUN
0,0,Jalisco
1,1,Acatic
2,2,Acatlán de Juárez
3,3,Ahualulco de Mercado
4,4,Amacueca


In [4]:
# unir dfs
df=df.merge(df_nombres, how='left', left_on='MUNICIPIO', right_on='CVE_MUN').drop('CVE_MUN', axis=1)

In [5]:
# info
print('n obs', df.shape)
df.info()

n obs (115589, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115589 entries, 0 to 115588
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ENTIDAD     115589 non-null  int64  
 1   MUNICIPIO   115589 non-null  int64  
 2   CODIGO      115230 non-null  float64
 3   ID_ESTRATO  59986 non-null   float64
 4   H001A       54867 non-null   float64
 5   NOM_MUN     115589 non-null  object 
dtypes: float64(3), int64(2), object(1)
memory usage: 5.3+ MB


Los datos utilizados solo son aquellos que tienen ID_ESTRATO sin valor

In [6]:
df=df.loc[df['ID_ESTRATO'].isna()]
df=df.drop('ID_ESTRATO', axis=1) # ya no es necesario ID ESTRATO 

In [7]:
df

,ENTIDAD,MUNICIPIO,CODIGO,H001A,NOM_MUN
0,14,0,NaN,2004175.0,Jalisco
5,14,0,11.0,8508.0,Jalisco
7,14,0,112.0,981.0,Jalisco
9,14,0,1125.0,981.0,Jalisco
11,14,0,11251.0,981.0,Jalisco
...,...,...,...,...,...
115579,14,125,812990.0,NaN,San Ignacio Cerro Gordo
115581,14,125,813.0,NaN,San Ignacio Cerro Gordo
115583,14,125,8132.0,NaN,San Ignacio Cerro Gordo
115585,14,125,81323.0,NaN,San Ignacio Cerro Gordo


In [8]:
# Formatear datos

df_filter=df.fillna(0) # reemplazar nan por 0

for col in df_filter.columns:
    if df_filter[col].dtype!='O':
        df_filter[col]=df_filter[col].astype(int) # cambiar formato a integer para todas las variables

In [9]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55603 entries, 0 to 115587
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ENTIDAD    55603 non-null  int64 
 1   MUNICIPIO  55603 non-null  int64 
 2   CODIGO     55603 non-null  int64 
 3   H001A      55603 non-null  int64 
 4   NOM_MUN    55603 non-null  object
dtypes: int64(4), object(1)
memory usage: 2.5+ MB


In [10]:
df_filter

,ENTIDAD,MUNICIPIO,CODIGO,H001A,NOM_MUN
0,14,0,0,2004175,Jalisco
5,14,0,11,8508,Jalisco
7,14,0,112,981,Jalisco
9,14,0,1125,981,Jalisco
11,14,0,11251,981,Jalisco
...,...,...,...,...,...
115579,14,125,812990,0,San Ignacio Cerro Gordo
115581,14,125,813,0,San Ignacio Cerro Gordo
115583,14,125,8132,0,San Ignacio Cerro Gordo
115585,14,125,81323,0,San Ignacio Cerro Gordo


## Numero de trabajadores por municipio
La variable H001A indica la cantidad de poblacion en el sector indicado por Codigo. Cuando codigo es igual a 0, eso indica la poblacion trabajando en todo el municipio

In [11]:
n_trabadores_municipio=df_filter[df_filter['CODIGO']==0][['MUNICIPIO', 'NOM_MUN', 'H001A']].copy()
n_trabadores_municipio

,MUNICIPIO,NOM_MUN,H001A
0,0,Jalisco,2004175
4866,1,Acatic,3086
5624,2,Acatlán de Juárez,3647
6502,3,Ahualulco de Mercado,2837
7474,4,Amacueca,214
...,...,...,...
111480,121,Zapotiltic,4749
112518,122,Zapotitlán de Vadillo,471
112937,123,Zapotlán del Rey,3943
113531,124,Zapotlanejo,11660


# FIltrar sectores en subclases economicas
La variable CODIGO indica la clase, subclase,etc, segun el rango de valores que adquiera.

Las subclases economicas (la denominacion mas especifica para una actividad economica) se encuentran en el rango [100000, 999999]

In [12]:
df_subclase=df_filter.drop(df_filter[df_filter['CODIGO'] <100000].index)
df_subclase=df_subclase.sort_values(by = ['MUNICIPIO', 'CODIGO'], na_position = 'first')
df_subclase

,ENTIDAD,MUNICIPIO,CODIGO,H001A,NOM_MUN
13,14,0,112511,56,Jalisco
15,14,0,112512,925,Jalisco
23,14,0,114112,0,Jalisco
25,14,0,114119,0,Jalisco
33,14,0,115111,0,Jalisco
...,...,...,...,...,...
115559,14,125,811493,0,San Ignacio Cerro Gordo
115567,14,125,812110,50,San Ignacio Cerro Gordo
115573,14,125,812310,0,San Ignacio Cerro Gordo
115579,14,125,812990,0,San Ignacio Cerro Gordo


# Matriz de Especializacion

In [13]:
matriz_trabajo=pd.pivot_table(df_subclase, values='H001A', index=['CODIGO'], columns=['MUNICIPIO'], aggfunc='max', fill_value=0, dropna=False)
matriz_trabajo

MUNICIPIO,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
CODIGO,,,,,,,,,,,,,,,,,,,,,
112511,56,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112512,925,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813110,2718,0,0,34,0,0,78,0,0,0,...,0,0,0,0,405,0,0,0,0,0
813120,2019,0,0,0,0,0,25,0,0,0,...,0,0,0,0,220,0,0,0,0,0
813130,4908,0,0,0,0,0,0,0,0,0,...,0,0,0,0,813,0,0,0,0,0


In [14]:
# convertir la tabla a un array para que sea mas facil realizar operacioens
matriz_trabajo_array=matriz_trabajo.to_numpy()

## Matriz de especializacion
El nivel de especializacion de un municipio se calcula con la formula
$$\text{Coeficiente Municipio}=\frac{\text{numero trabadores en sector}}{\text{trabajores totales en municipio}}$$

In [15]:

matriz_especializacion = matriz_trabajo_array / n_trabadores_municipio['H001A'].to_numpy()

matriz_especializacion=pd.DataFrame(matriz_especializacion, index=df_subclase['CODIGO'].unique(), columns=df_nombres['NOM_MUN'].values)
matriz_especializacion.index.rename('CODIGO', inplace=True)
matriz_especializacion

,Jalisco,Acatic,Acatlán de Juárez,Ahualulco de Mercado,Amacueca,Amatitán,Ameca,San Juanito de Escobedo,Arandas,El Arenal,...,Villa Hidalgo,Cañadas de Obregón,Yahualica de González Gallo,Zacoalco de Torres,Zapopan,Zapotiltic,Zapotitlán de Vadillo,Zapotlán del Rey,Zapotlanejo,San Ignacio Cerro Gordo
CODIGO,,,,,,,,,,,,,,,,,,,,,
112511,0.000028,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
112512,0.000462,0.017822,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
114112,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
114119,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
115111,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813110,0.001356,0.000000,0.000000,0.011984,0.0,0.0,0.008318,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000979,0.0,0.0,0.0,0.0,0.0
813120,0.001007,0.000000,0.000000,0.000000,0.0,0.0,0.002666,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000532,0.0,0.0,0.0,0.0,0.0
813130,0.002449,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.001966,0.0,0.0,0.0,0.0,0.0


# Matriz de Coeficientes de especializacion
Esta matriz muestra el nivel de especializacion un sector en un minicipio considerando el grado de especializacion en todo Estado. 

$$\text{Coeficiente Especializacion}=\frac{\text{Coeficiente especializacion del sector en el municipio}}{\text{Coeficiente especializacion en el Estado}}$$

In [19]:
coeff_estado=matriz_especializacion.iloc[:, 0]

coeff_especializacion=np.divide(matriz_especializacion.iloc[:, 1:].T, coeff_estado).T
coeff_especializacion=coeff_especializacion.fillna(0)
coeff_especializacion=coeff_especializacion.replace(np.inf, 0)
coeff_especializacion

,Acatic,Acatlán de Juárez,Ahualulco de Mercado,Amacueca,Amatitán,Ameca,San Juanito de Escobedo,Arandas,El Arenal,Atemajac de Brizuela,...,Villa Hidalgo,Cañadas de Obregón,Yahualica de González Gallo,Zacoalco de Torres,Zapopan,Zapotiltic,Zapotitlán de Vadillo,Zapotlán del Rey,Zapotlanejo,San Ignacio Cerro Gordo
CODIGO,,,,,,,,,,,,,,,,,,,,,
112511,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
112512,38.615412,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
114112,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
114119,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
115111,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813110,0.000000,0.000000,8.837019,0.0,0.0,6.133620,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.722229,0.0,0.0,0.0,0.0,0.0
813120,0.000000,0.000000,0.000000,0.0,0.0,2.646521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.528148,0.0,0.0,0.0,0.0,0.0
813130,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.802889,0.0,0.0,0.0,0.0,0.0


In [20]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(100, 100))
sns.heatmap(coeff_especializacion.T, vmin=0);

In [37]:
coeff_especializacion[coeff_especializacion > 0].describe().loc['75%'].mean()

np.float64(3.3730212877080072)

In [21]:
coeff_especializacion.to_csv(MATRICES_DIR, index=True,mode='w')